# Deployment Guardrails to update Endpoint
---

## Introduction
---

SageMaker 배포 가드레일(Deployment Guardrail)은 프로덕션 환경에서 현재 모델에서 새 모델로 안전하게 업데이트하기 위한 완전 관리형 블루/그린 배포 가드레일 서비스입니다.
카나리 및 선형과 같은 트래픽 전환 모드를 사용하여 업데이트 과정에서 현재 모델에서 새 모델로 트래픽 전환 프로세스를 세부적으로 제어할 수 있습니다. 또한 문제를 조기에 포착하고 프로덕션에 영향을 미치지 않게 자동 롤백과 같은 보호 기능을 제공합니다.

트래픽 전환 모드는 엔드포인트 트래픽이 업데이트가 포함된 새 집합으로 라우팅되는 방식을 지정하는 구성으로, 엔드포인트 업데이트 프로세스에 대한 다양한 제어 수준을 제공합니다.

- **All-At-Once Traffic Shifting** : 모든 엔드포인트 트래픽을 블루 플릿에서 그린 플릿으로 전환합니다. 트래픽이 그린 플릿으로 이동하면 미리 지정된 Amazon CloudWatch 알람이 설정된 시간("베이킹 기간; baking period") 동안 그린 플릿 모니터링을 시작합니다. 베이킹 기간 동안 알람이 트리거되지 않으면 블루 플릿이 종료됩니다.
- **Canary Traffic Shifting** : 트래픽의 작은 부분("카나리")을 그린 플릿으로 이동하고 베이킹 기간 동안 모니터링합니다. 카나리 배포가 그린 플릿에서 성공하면 나머지 트래픽은 블루 플릿을 종료하기 전에 블루 플릿에서 그린 플릿으로 이동합니다.
- **Linear Traffic Shifting**: 트래픽 이동 단계를 n개로 확장하여 각 단계에 대해 이동할 트래픽 비율에 대해 세부적으로 지정할 수 있습니다.

본 실습에서는 트래픽 이동 및 자동 롤백 기능을 보여주기 위해 아래와 같은 기능들을 체험해 봅니다.

- 모델 1~모델 3에 대한 모델 및 엔드포인트 구성 생성
    - 모델 1: 정상 동작, PyTorch 1.7.1
    - 모델 2: 에러 발생, PyTorch 1.7.1 
    - 모델 3: 정상 동작, PyTorch 1.8.1
- 모델 1의 엔드포인트 설정으로 엔드포인트 시작
- 롤백을 트리거하는 데 사용되는 CloudWatch 알람 지정
- 모델 2의 엔드포인트 설정을 가리키도록 엔드포인트 업데이트 
    - 일정 시간 경과 후 일부 트래픽이 모델 2를 호출하는 플릿으로 이동되며, CloudWatch 알람에서 오류 이벤트를 감지하여 자동으로 모델 1 플릿으로 롤백 
- 모델 3의 엔드포인트 설정을 가리키도록 엔드포인트 업데이트 
    - 일정 시간 경과 후 일부 트래픽이 모델 3을 호출하는 플릿으로 이동되며, 오류가 없다면 점진적으로 모든 트래픽이 모델 3 플릿으로 이동

### References
- Take advantage of advanced deployment strategies using Amazon SageMaker deployment guardrails: https://aws.amazon.com/ko/blogs/machine-learning/take-advantage-of-advanced-deployment-strategies-using-amazon-sagemaker-deployment-guardrails/

In [ ]:
import os
import json
import sys
import logging
import boto3
import sagemaker
import time
from datetime import datetime, timedelta
from sagemaker.huggingface import HuggingFaceModel
from sagemaker import session
from transformers import ElectraConfig
from transformers import (
    ElectraModel, ElectraTokenizer, ElectraForSequenceClassification
)
from src.utils import print_outputs, upload_model_artifact_to_s3, NLPPredictor 


logging.basicConfig(
    level=logging.INFO, 
    format='[{%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(filename='tmp.log'),
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sess = sagemaker.Session()
boto_session = boto3.session.Session()
sm_client = boto_session.client("sagemaker")
sm_runtime = boto3.Session().client("sagemaker-runtime")

<br>

## 1. Create and Deploy Models
---

사전 훈련된 한국어 자연어 처리 모델(네이버 감성 분류 긍정/부정 판별)을 배포합니다.

In [ ]:
bucket = sess.default_bucket()
prefix = 'deployment-guardraril-kornlp-nsmc'

모델 파라메터, 토크나이저와 추론 코드를 `model.tar.gz`으로 압축하여 S3로 복사합니다. 압축 파일명은 자유롭게 지정할 수 있으나, 반드시 `tar.gz`로 압축해야 합니다.

In [ ]:
model_variant = 'modelA'
nlp_task = 'nsmc'
model_path = f'model-{nlp_task}'
model_s3_uri = upload_model_artifact_to_s3(model_variant, model_path, bucket, prefix)

### Create Models

In [ ]:
from sagemaker.image_uris import retrieve

ecr_image_uri1 = retrieve(
    framework='pytorch',
    region=region,
    version='1.7.1',
    py_version='py3',
    instance_type='ml.m5.xlarge',
    accelerator_type=None,
    image_scope='inference'
)

ecr_image_uri2 = retrieve(
    framework='pytorch',
    region=region,
    version='1.7.1',
    py_version='py3',
    instance_type='ml.m5.xlarge',
    accelerator_type=None,
    image_scope='inference'
)

ecr_image_uri3 = retrieve(
    framework='pytorch',
    region=region,
    version='1.8.1',
    py_version='py3',
    instance_type='ml.m5.xlarge',
    accelerator_type=None,
    image_scope='inference'
)

print(f"Model Image 1: {ecr_image_uri1}")
print(f"Model Image 2: {ecr_image_uri2}")
print(f"Model Image 3: {ecr_image_uri3}")

`inference_nsmc_error.py`은 인위적으로 에러를 발생하기 위해 '12'/'34'의 더미 코드를 삽입했습니다.

In [ ]:
model_name1 = f"model1-{prefix}-{datetime.now():%Y-%m-%d-%H-%M-%S}"
model_name2 = f"model2-{prefix}-{datetime.now():%Y-%m-%d-%H-%M-%S}"
model_name3 = f"model3-{prefix}-{datetime.now():%Y-%m-%d-%H-%M-%S}"

resp1 = sm_client.create_model(
    ModelName=model_name1,
    Containers=[
        {
            "Image": ecr_image_uri1,
            "Mode": "SingleModel",
            "ModelDataUrl": model_s3_uri,
            "Environment": {
                "SAGEMAKER_CONTAINER_LOG_LEVEL": "20",
                "SAGEMAKER_PROGRAM": "inference_nsmc.py",
                "SAGEMAKER_SUBMIT_DIRECTORY": model_s3_uri,
            },                
        }        
        
    ],
    ExecutionRoleArn=role,
)

resp2 = sm_client.create_model(
    ModelName=model_name2,
    Containers=[
        {
            "Image": ecr_image_uri2,
            "Mode": "SingleModel",
            "ModelDataUrl": model_s3_uri,
            "Environment": {
                "SAGEMAKER_CONTAINER_LOG_LEVEL": "20",
                "SAGEMAKER_PROGRAM": "inference_nsmc_error.py",
                "SAGEMAKER_SUBMIT_DIRECTORY": model_s3_uri,
            },                
        }        
        
    ],
    ExecutionRoleArn=role,
)

resp3 = sm_client.create_model(
    ModelName=model_name3,
    Containers=[
        {
            "Image": ecr_image_uri3,
            "Mode": "SingleModel",
            "ModelDataUrl": model_s3_uri,
            "Environment": {
                "SAGEMAKER_CONTAINER_LOG_LEVEL": "20",
                "SAGEMAKER_PROGRAM": "inference_nsmc.py",
                "SAGEMAKER_SUBMIT_DIRECTORY": model_s3_uri,
            },                
        }        
        
    ],
    ExecutionRoleArn=role,
)

print(f"Created Model1: {resp1['ModelArn']}")      
print(f"Created Model2: {resp2['ModelArn']}")
print(f"Created Model3: {resp3['ModelArn']}") 

### Create Endpoint Configs

In [ ]:
endpoint_config_name1 = f"endpoint-config1-{datetime.now():%Y-%m-%d-%H-%M-%S}"
endpoint_config_name2 = f"endpoint-config2-{datetime.now():%Y-%m-%d-%H-%M-%S}"
endpoint_config_name3 = f"endpoint-config3-{datetime.now():%Y-%m-%d-%H-%M-%S}"

config_resp1 = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name1,
    ProductionVariants=[
        {
            "VariantName": "AllTraffic",
            "InstanceType": "ml.m5.xlarge",
            "InitialInstanceCount": 3,
            "ModelName": model_name1,          
        },
    ],
)

config_resp2 = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name2,
    ProductionVariants=[
        {
            "VariantName": "AllTraffic",
            "InstanceType": "ml.m5.xlarge",
            "InitialInstanceCount": 3,
            "ModelName": model_name2,          
        },
    ],
)

config_resp3 = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name3,
    ProductionVariants=[
        {
            "VariantName": "AllTraffic",
            "InstanceType": "ml.m5.xlarge",
            "InitialInstanceCount": 3,
            "ModelName": model_name3,          
        },
    ],
)

print(f"Created EndpointConfig1: {config_resp1['EndpointConfigArn']}")
print(f"Created EndpointConfig2: {config_resp2['EndpointConfigArn']}")
print(f"Created EndpointConfig3: {config_resp3['EndpointConfigArn']}")  

### Create Endpoint

엔드포인트 설정1(베이스라인)에 대해 호스팅 엔드포인트를 배포합니다.

In [ ]:
endpoint_name = f"endpoint-canary-{nlp_task}-{datetime.now():%Y-%m-%d-%H-%M-%S}"
endpoint_resp = sm_client.create_endpoint(
    EndpointName=endpoint_name, 
    EndpointConfigName=endpoint_config_name1
)

print(f"Creating Endpoint: {endpoint_resp['EndpointArn']}")

In [ ]:
from IPython.core.display import display, HTML

def make_endpoint_link(region, endpoint_name, endpoint_task):
    endpoint_link = f'<b><a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={region}#/endpoints/{endpoint_name}">{endpoint_task} Review Endpoint</a></b>'   
    return endpoint_link 

def wait_for_endpoint_in_service(endpoint_name):
    waiter = boto3.client('sagemaker').get_waiter('endpoint_in_service')
    print("Waiting for endpoint to create...")
    waiter.wait(EndpointName=endpoint_name)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    print(f"Endpoint Status: {resp['EndpointStatus']}")
        
endpoint_link = make_endpoint_link(region, endpoint_name, '[Deploy model from S3]')
display(HTML(endpoint_link))

In [ ]:
sm_client.describe_endpoint(EndpointName=endpoint_name)
wait_for_endpoint_in_service(endpoint_name)

<br>

## 2. Invoke Endpoint
---

엔드포인트 배포가 완료되었다면 실시간 추론을 수행할 수 있습니다. 테스트를 위해 최대 호출 횟수(maximum invocations) 및 대기 간격(waiting interval)을 지정하여 엔드포인트를 여러 번 호출합니다.

In [ ]:
sample_file = f'sample_{nlp_task}.txt'
with open(sample_file, mode='wt', encoding='utf-8') as f:
    f.write('{"text": ["이 영화는 최고의 영화입니다"]}\n')    
    f.write('{"text": ["최악이에요. 배우의 연기력도 좋지 않고 내용도 너무 허접합니다"]}')
    
with open(sample_file, mode='rb') as f:
    payloads = f.read()  

In [ ]:
from datetime import datetime, timedelta, timezone

def invoke_endpoint(payloads, endpoint_name, target_variant=None):
    start = time.time()
    if target_variant is not None:
        response = sm_runtime.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType="application/jsonlines", 
            Accept="application/jsonlines",            
            TargetVariant=target_variant,
            Body=payloads,
        )        
    else:        
        response = sm_runtime.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType="application/jsonlines", 
            Accept="application/jsonlines",                   
            Body=payloads,
        )
    latency = (time.time() - start) * 1000
    variant = response["InvokedProductionVariant"]
    logger.info(f'[{variant}] Latency: {latency:.3f} ms')
    
    outputs = response['Body'].read().decode()
    return outputs

def invoke_endpoint_many(payloads, endpoint_name, num_requests=250, sleep_secs=0.5, should_raise_exp=False):
    for i in range(num_requests):
        try:
            response = sm_runtime.invoke_endpoint(
                EndpointName=endpoint_name,
                ContentType="application/jsonlines", 
                Accept="application/jsonlines", 
                Body=payloads,
            )
            outputs = response['Body'].read().decode()
            print(".", end="", flush=True)
        except Exception as e:
            print("E", end="", flush=True)    
            if should_raise_exp:
                raise e

        time.sleep(sleep_secs)
    print('\nDone!')

In [ ]:
outputs = invoke_endpoint(payloads, endpoint_name)
print_outputs(outputs)

In [ ]:
invocation_start_time = datetime.now()
invoke_endpoint_many(payloads, endpoint_name, 250, 0.5)
time.sleep(20)  # give metrics time to catch up

### Invocations Metrics

Amazon SageMaker는 Amazon CloudWatch를 쿼리하여 레이턴시 및 호출(invocations)과 같은 지표들을 모니터링할 수 있습니다. 모니터링 가능한 지표들은 아래 링크를 참조해 주세요.
- https://docs.aws.amazon.com/sagemaker/latest/dg/monitoring-cloudwatch.html

In [ ]:
import pandas as pd

cw = boto3.Session().client("cloudwatch", region_name=region)

def get_sagemaker_metrics(
    endpoint_name,
    endpoint_config_name,
    variant_name,
    metric_name,
    statistic,
    start_time,
    end_time,
):
    dimensions = [
        {"Name": "EndpointName", "Value": endpoint_name},
        {"Name": "VariantName", "Value": variant_name},
    ]
    if endpoint_config_name is not None:
        dimensions.append({"Name": "EndpointConfigName", "Value": endpoint_config_name})
    metrics = cw.get_metric_statistics(
        Namespace="AWS/SageMaker",
        MetricName=metric_name,
        StartTime=start_time,
        EndTime=end_time,
        Period=60,
        Statistics=[statistic],
        Dimensions=dimensions,
    )
    rename = endpoint_config_name if endpoint_config_name is not None else "ALL"
    if len(metrics["Datapoints"]) == 0:
        return
    return (
        pd.DataFrame(metrics["Datapoints"])
        .sort_values("Timestamp")
        .set_index("Timestamp")
        .drop(["Unit"], axis=1)
        .rename(columns={statistic: rename})
    )


def plot_endpoint_invocation_metrics(
    endpoint_name,
    endpoint_config_name,
    variant_name,
    metric_name,
    statistic,
    start_time=None,
):
    start_time = start_time or datetime.now(timezone.utc) - timedelta(minutes=60)
    end_time = datetime.now(timezone.utc)
    metrics_variants = get_sagemaker_metrics(
        endpoint_name,
        endpoint_config_name,
        variant_name,
        metric_name,
        statistic,
        start_time,
        end_time,
    )
    if metrics_variants is None:
        return
    metrics_variants.plot(title=f"{metric_name}-{statistic}")
    return metrics_variants

### Plot endpoint invocation metrics

아래 코드 셀을 호출하면, 엔드포인트에 대한 Invocation,Invocation4XXErrors,Invocation5XXErrors,ModelLatency 및 OverheadLatency를 표시하는 그래프를 플롯합니다.

현재는 정상적으로 동작하는 모델 버전과 설정을 사용하고 있으므로 Invocation4XXErrors 및 Invocation5XXErrors는 플랫 라인(y축의 값이 일정하게 0)에 있어야 함을 알 수 있습니다. 또한 ModelLatency 및 OverheadLatency는 시간이 지남에 따라 감소하기 시작합니다.

In [ ]:
invocation_metrics = plot_endpoint_invocation_metrics(
    endpoint_name, endpoint_config_name1, "AllTraffic", "Invocations", "Sum", invocation_start_time
)
invocation_4xx_metrics = plot_endpoint_invocation_metrics(
    endpoint_name, None, "AllTraffic", "Invocation4XXErrors", "Sum", invocation_start_time
)
invocation_5xx_metrics = plot_endpoint_invocation_metrics(
    endpoint_name, None, "AllTraffic", "Invocation5XXErrors", "Sum", invocation_start_time
)
model_latency_metrics = plot_endpoint_invocation_metrics(
    endpoint_name, None, "AllTraffic", "ModelLatency", "Average", invocation_start_time
)
overhead_latency_metrics = plot_endpoint_invocation_metrics(
    endpoint_name, None, "AllTraffic", "OverheadLatency", "Average", invocation_start_time
)

<br>

## 3. Create CloudWatch alarms to monitor Endpoint performance
---

본 섹션에서는 아래 지표들을 사용하여 엔드포인트 성능을 모니터링하는 CloudWatch 알람을 생성합니다.
* Invocation5XXErrors
* ModelLatency

CloudWatch 알람을 생성하는 메소드(`put_metric_alarm`)의 인자값에서 Dimension은 엔드포인트 설정 및 variant별로 지표를 선택하는 데 사용됩니다. 
* EndpointName
* VariantName

In [ ]:
def create_auto_rollback_alarm(
    alarm_name, endpoint_name, variant_name, metric_name, statistic, threshold
):
    cw.put_metric_alarm(
        AlarmName=alarm_name,
        AlarmDescription="Test SageMaker endpoint deployment auto-rollback alarm",
        ActionsEnabled=False,
        Namespace="AWS/SageMaker",
        MetricName=metric_name,
        Statistic=statistic,
        Dimensions=[
            {"Name": "EndpointName", "Value": endpoint_name},
            {"Name": "VariantName", "Value": variant_name},
        ],
        Period=60,
        EvaluationPeriods=1,
        Threshold=threshold,
        ComparisonOperator="GreaterThanOrEqualToThreshold",
        TreatMissingData="notBreaching",
    )

In [ ]:
error_alarm = f"TestAlarm-5XXErrors-{endpoint_name}"
latency_alarm = f"TestAlarm-ModelLatency-{endpoint_name}"

# alarm on 5xx error rate for 1 minute
create_auto_rollback_alarm(
    error_alarm, endpoint_name, "AllTraffic", "Invocation5XXErrors", "Average", 0.1
)
# alarm on model latency >= 400 ms for 1 minute
create_auto_rollback_alarm(
    latency_alarm, endpoint_name, "AllTraffic", "ModelLatency", "Average", 400000
)

In [ ]:
cw.describe_alarms(AlarmNames=[error_alarm, latency_alarm])
time.sleep(60)

<br>

## 4. Update Endpoint with deployment configurations
---

엔드포인트를 업데이트하고 CloudWatch 지표에서 성능을 모니터링합니다.

### BlueGreen update policy with Canary/Linear traffic shifting

트래픽이 이전 스택에서 새 스택으로 이동하는 블루/그린 업데이트를 쉽게 수행할 수 있습니다. 카나리(Canary)/선형(Linear) 모드를 사용하면 호출 요청이 신규 버전의 모델로 점진적으로 이동하여 오류가 트래픽의 100%에 영향을 미치는 것을 방지합니다. 새 버전의 모델에서 일정 이상의 오류 발생 시 자동으로 이전 버전의 모델로 롤백함으로써, 
신규 버전 모델 배포에 대한 리스크를 최소화할 수 있습니다. 

### Rollback Case 

호환되지 않는 모델 버전(모델-2, 엔드포인트 config-2)으로 엔드포인트를 업데이트하여 오류를 시뮬레이션하고 롤백을 트리거합니다.

In [ ]:
# canary_deployment_config
canary_deployment_config = {
    "BlueGreenUpdatePolicy": {
        "TrafficRoutingConfiguration": {
            "Type": "CANARY",
            "CanarySize": {
                "Type": "INSTANCE_COUNT",  # or use "CAPACITY_PERCENT" as 30%, 50%
                "Value": 1,
            },
            "WaitIntervalInSeconds": 300,  # wait for 5 minutes before enabling traffic on the rest of fleet
        },
        "TerminationWaitInSeconds": 120,  # wait for 2 minutes before terminating the old stack
        "MaximumExecutionTimeoutInSeconds": 1800,  # maximum timeout for deployment
    },
    "AutoRollbackConfiguration": {
        "Alarms": [{"AlarmName": error_alarm}],
    },
}

# linear_deployment_config
linear_deployment_config = {
    "BlueGreenUpdatePolicy": {
        "TrafficRoutingConfiguration": {
            "Type": "LINEAR",
            "LinearStepSize": {
                "Type": "CAPACITY_PERCENT",
                "Value": 33,  # 33% of whole fleet capacity (33% * 3 = 1 instance)
            },
            "WaitIntervalInSeconds": 180,  # wait for 3 minutes before enabling traffic on the rest of fleet
        },
        "TerminationWaitInSeconds": 120,  # wait for 2 minutes before terminating the old stack
        "MaximumExecutionTimeoutInSeconds": 1800,  # maximum timeout for deployment
    },
    "AutoRollbackConfiguration": {
        "Alarms": [{"AlarmName": error_alarm}],
    },
}

# update endpoint request with new DeploymentConfig parameter
sm_client.update_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name2,
    DeploymentConfig=linear_deployment_config,
)

In [ ]:
sm_client.describe_endpoint(EndpointName=endpoint_name)

### Invoke the endpoint during the update operation is in progress

아래 코드 셀을 실행하면 카나리/선형 플릿의 오류를 시뮬레이션합니다. 일정 시간 경과 후 확률적으로 오류(E)가 표시됩니다.

In [ ]:
invoke_endpoint_many(payloads, endpoint_name, 400, 0.8)

엔드포인트 업데이트 작업이 완료될 때까지 기다렸다가 자동 롤백을 확인합니다.

In [ ]:
wait_for_endpoint_in_service(endpoint_name)
sm_client.describe_endpoint(EndpointName=endpoint_name)

이전 모델로 다시 롤백되어서 추론이 잘 이루어지고 있음을 확인할 수 있습니다. 만약 엔드포인트

In [ ]:
outputs = invoke_endpoint(payloads, endpoint_name)
print_outputs(outputs)

아래 코드 셀을 실행하면 Invocations,Invocation5XXErrors 및 ModelLatency를 엔드포인트에 대해 표시하는 그래프를 플롯합니다.

신규 엔드포인트 config-2(오류가 발생하는 모델-2)로 엔드포인트를 업데이트하면, 일정 시간 경과 후 CloudWatch 알람이 발생하고 엔드포인트 config-1로 롤백됩니다. 이 롤백 단계에서 Invocation5XXErrors가 증가하는 것을 아래 그래프에서 볼 수 있습니다.

In [ ]:
invocation_metrics = plot_endpoint_invocation_metrics(
    endpoint_name, None, "AllTraffic", "Invocations", "Sum"
)
metrics_epc_1 = plot_endpoint_invocation_metrics(
    endpoint_name, endpoint_config_name1, "AllTraffic", "Invocations", "Sum"
)
metrics_epc_2 = plot_endpoint_invocation_metrics(
    endpoint_name, endpoint_config_name2, "AllTraffic", "Invocations", "Sum"
)

metrics_all = invocation_metrics.join([metrics_epc_1, metrics_epc_2], how="outer")
metrics_all.plot(title="Invocations-Sum")

invocation_5xx_metrics = plot_endpoint_invocation_metrics(
    endpoint_name, None, "AllTraffic", "Invocation5XXErrors", "Sum"
)
model_latency_metrics = plot_endpoint_invocation_metrics(
    endpoint_name, None, "AllTraffic", "ModelLatency", "Average"
)

### Success Case

이번에는 동일한 카나리/선형 배포 설정을 사용하지만 유효한 엔드포인트 설정을 사용하는 성공 사례를 살펴보겠습니다.

먼저, 정상적으로 동작하는 엔드포인트 config-3으로 엔드포인트를 업데이트합니다.

In [ ]:
# update endpoint request with new DeploymentConfig parameter
sm_client.update_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name3,
    RetainDeploymentConfig=True,
)

아래 코드 셀을 실행하면 카나리/선형 플릿을 시뮬레이션합니다. 오류가 발생하지 않는 모델이므로 정상적으로 수행됩니다.

In [ ]:
invoke_endpoint_many(payloads, endpoint_name, 300, 0.8)

In [ ]:
wait_for_endpoint_in_service(endpoint_name)
sm_client.describe_endpoint(EndpointName=endpoint_name)

신규 엔드포인트 config-3(올바르게 동작하는 모델-3)으로 엔드포인트를 업데이트하면, 오류 없이 엔드포인트 config-2(오류가 발생하는 모델-2)를 인수합니다. 이 전환 단계에서 Invocation5XXErrors가 감소하는 것을 아래 그래프에서 볼 수 있습니다.

In [ ]:
invocation_metrics = plot_endpoint_invocation_metrics(
    endpoint_name, None, "AllTraffic", "Invocations", "Sum"
)
metrics_epc_1 = plot_endpoint_invocation_metrics(
    endpoint_name, endpoint_config_name1, "AllTraffic", "Invocations", "Sum"
)
metrics_epc_2 = plot_endpoint_invocation_metrics(
    endpoint_name, endpoint_config_name2, "AllTraffic", "Invocations", "Sum"
)
metrics_epc_3 = plot_endpoint_invocation_metrics(
    endpoint_name, endpoint_config_name3, "AllTraffic", "Invocations", "Sum"
)

In [ ]:
metrics_all = invocation_metrics.join([metrics_epc_1, metrics_epc_2, metrics_epc_3], how="outer")
metrics_all.plot(title="Invocations-Sum")

invocation_5xx_metrics = plot_endpoint_invocation_metrics(
    endpoint_name, None, "AllTraffic", "Invocation5XXErrors", "Sum"
)
model_latency_metrics = plot_endpoint_invocation_metrics(
    endpoint_name, None, "AllTraffic", "ModelLatency", "Average"
)

<br>

## Clean up
---

In [ ]:
sm_client.delete_endpoint(EndpointName=endpoint_name)

In [ ]:
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name1)
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name2)
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name3)

In [ ]:
sm_client.delete_model(ModelName=model_name1)
sm_client.delete_model(ModelName=model_name2)
sm_client.delete_model(ModelName=model_name3)

In [ ]:
cw.delete_alarms(AlarmNames=[error_alarm, latency_alarm])